In [1]:
import sys
sys.path.append("../") # go to parent dir

In [2]:
from src.FinateStateMachine import FSM

In [3]:
from src.compound_analyzer import CompoundAnalyzer

In [4]:
roots = []
with open("../data/rusgram_roots.txt") as f:
    for l in f:
        l = l.split()[0]
        l = "".join(list(filter(lambda x: x in "-ёйцукенгшщзхъфывапролджэячсмитьбю", l)))
        roots.append(l)
roots = sorted(set(roots))

In [5]:
roots_r = list(map(lambda x: x.replace('-', ''), filter(lambda x: x.startswith('-'), roots)))
roots_l = list(map(lambda x: x.replace('-', ''), filter(lambda x: x.endswith('-'), roots)))

In [6]:
print(roots_l)

['авиа', 'авто', 'агло', 'агро', 'аква', 'аллерго', 'англо', 'анти', 'антропо', 'арифмо', 'астро', 'атто', 'аудио', 'аэро', 'баро', 'бензо', 'библио', 'био', 'блиц', 'вело', 'верто', 'вибро', 'видео', 'гальвано', 'гекто', 'гелио', 'гео', 'гермо', 'гига', 'гигро', 'гидро', 'гипо', 'графо', 'дека', 'дендро', 'деци', 'диапо', 'зепто', 'зетта', 'зоо', 'изо', 'иммуно', 'иокто', 'иотта', 'кардио', 'квази', 'кило', 'космо', 'кримино', 'макро', 'макси', 'мега', 'метео', 'микро', 'милли', 'мини', 'моно', 'мото', 'нано', 'нарко', 'невро', 'нейро', 'нео', 'нитро', 'пета', 'пико', 'пиро', 'пневмо', 'поли', 'псевдо', 'психо', 'радио', 'русо', 'санти', 'сейсмо', 'сексо', 'социо', 'стерео', 'страто', 'теле', 'тера', 'термо', 'техно', 'типо', 'троллей', 'турбо', 'ультра', 'фемто', 'физио', 'фоно', 'фото', 'хромо', 'хроно', 'эвако', 'экса', 'электро', 'энерго']


In [7]:
print(roots_r)

['бол', 'бус', 'гамия', 'грамма', 'граф', 'дром', 'кар', 'лиз', 'лог', 'локация', 'ман', 'мер', 'метр', 'мобиль', 'навт', 'пат', 'план', 'рама', 'роб', 'скоп', 'стан', 'стат', 'тека', 'трон', 'фил', 'фоб', 'фон']


# F-F

In [45]:
foreign_L = FSM({"FOREIGN-L"})
foreign_R = FSM({"FOREIGN-R"})

for word in roots_l:
    foreign_L.add_word(
        list(word) + ["FOREIGN-L", (word, "FOREIGN-L")]
    )
    
for word in roots_r:
    foreign_R.add_word(
        list(word) + ["FOREIGN-R", (word, "FOREIGN-R")]
    )

In [46]:
foreign_ia_R = FSM({"rule578*(noun + иj(а) -> noun)"})

for word in roots_r:
    foreign_ia_R.add_word(
        list(word + "ия") + ["rule578*(noun + иj(а) -> noun)", (word, "FOREIGN-R")]
    )

In [47]:
cas = {}

In [48]:
ff_rule = "rule550(FOREIGN + FOREIGN -> noun)"

ca_ff = CompoundAnalyzer(ff_rule, "noun", foreign_L, foreign_R)
cas[ff_rule] = ca_ff

In [49]:
noun_R = FSM({"noun"})

nouns = []
for l in open("../data/wiktionary/noun.txt"):
    l = l.strip()
    nouns.append(l)
    noun_R.add_word(
        list(l.lower()) + ["noun", (l, "noun")]
    )

# F-N

В качестве левого автомата использовать foreign_L, а качестве правого --- noun.

In [50]:
fn_rule = "rule550(FOREIGN + noun -> noun)"


ca_fn = CompoundAnalyzer(fn_rule, "noun", foreign_L, noun_R)
cas[fn_rule] = ca_fn

# 558

In [51]:
ffia_rule = "rule578(FOREIGN + FOREIGN + иj(а) -> noun)"
cas[ffia_rule] = CompoundAnalyzer(ffia_rule, "noun", foreign_L, foreign_ia_R)

# Finally

In [52]:
cas

{'rule550(FOREIGN + FOREIGN -> noun)': <src.compound_analyzer.CompoundAnalyzer at 0x7f1ca65c4438>,
 'rule550(FOREIGN + noun -> noun)': <src.compound_analyzer.CompoundAnalyzer at 0x7f1ca65c4048>,
 'rule578(FOREIGN + FOREIGN + иj(а) -> noun)': <src.compound_analyzer.CompoundAnalyzer at 0x7f1ca65c42b0>}

In [53]:
def analyze_word(word, pos):
    analyzes = []
    for rule_name, ca in cas.items():
        if ca.pos != pos:
            continue        
        analyzes.extend(ca.analyze(word, pos))
    return analyzes

In [54]:
analyze_word("астролог", "noun")

[('астролог',
  'noun',
  'rule550(FOREIGN + FOREIGN -> noun)',
  ('астро', 'FOREIGN-L'),
  'FOREIGN-L',
  ('лог', 'FOREIGN-R'),
  'FOREIGN-R'),
 ('астролог',
  'noun',
  'rule550(FOREIGN + noun -> noun)',
  ('астро', 'FOREIGN-L'),
  'FOREIGN-L',
  ('лог', 'noun'),
  'noun'),
 ('астролог',
  'noun',
  'rule550(FOREIGN + noun -> noun)',
  ('астро', 'FOREIGN-L'),
  'FOREIGN-L',
  ('Лог', 'noun'),
  'noun')]

In [55]:
analyze_word("аквадискотека", "noun")

[('аквадискотека',
  'noun',
  'rule550(FOREIGN + noun -> noun)',
  ('аква', 'FOREIGN-L'),
  'FOREIGN-L',
  ('дискотека', 'noun'),
  'noun')]

In [56]:
analyze_word("астрофобия", "noun")

[('астрофобия',
  'noun',
  'rule550(FOREIGN + noun -> noun)',
  ('астро', 'FOREIGN-L'),
  'FOREIGN-L',
  ('фобия', 'noun'),
  'noun'),
 ('астрофобия',
  'noun',
  'rule578(FOREIGN + FOREIGN + иj(а) -> noun)',
  ('астро', 'FOREIGN-L'),
  'FOREIGN-L',
  ('фоб', 'FOREIGN-R'),
  'rule578*(noun + иj(а) -> noun)')]